In [2]:
import sys
#sys.path.append('/Users/antonvoskobovich/IdeaProjects/TitanicSurvivals2')

import importlib

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np

from tqdm import tqdm

In [4]:
!ls -l

total 3264
-rw-r--r-- 1 jovyan users     900 Mar 29 22:37 a.ipynb
-rw-r--r-- 1 jovyan users  159794 Jun 18 19:05 evernote_.ipynb
-rw-r--r-- 1 jovyan users 2057068 Apr 19 13:57 notes.gexf
-rw-r--r-- 1 jovyan users 1066751 Apr 19 13:55 notes.graphml
-rw-r--r-- 1 jovyan users       0 Apr 19 13:55 notes_main.graphml
-rw-r--r-- 1 jovyan users    1854 Apr 19 08:01 x.csv
-rw-r--r-- 1 jovyan users      65 May 12 17:28 x.py
-rw-r--r-- 1 jovyan users    1829 Apr 19 08:03 x2.csv


In [5]:
context_dir = '../../work/data/full/'
#context_dir = 'work/data/full/'

In [6]:
import os
os.getcwd()

'/home/jovyan/work/notes_analytics'

## EDA

In [16]:
df = pd.read_parquet(f'{context_dir}/raw_notes')

In [17]:
df.groupby('stack')['stack'].count()

stack
Archive               1053
Areas                  567
Core                   331
Data                   303
Debug                    4
Life Mapping Stack     204
Maps                   853
Operations             829
Personal Projects      503
Professional           532
Reflections           1950
Simple                 352
Techs                  288
Name: stack, dtype: int64

In [ ]:
df.groupby('stack')['contentLength'].mean()

In [ ]:
df.head()

In [ ]:
df.query('id == "9e390b86-b145-4436-9d50-b05b372a4c4d"')

In [ ]:
#sns.jointplot(x=np.log1p(df['title'].str.len()),  y=np.log1p(df['contentLength']))

## Количество заметок созданных в месяц

In [ ]:
df['created_month'] = df['created'].dt.to_period('M').dt.start_time
monthes_cumsum = df['created_month'].value_counts().sort_index().cumsum()
monthes_sum = df.groupby('created_month').agg(created_notes = ('id', 'count')).sort_index()

In [ ]:
monthes_sum.iloc[-20:]

In [ ]:
sns.lineplot(data=monthes_sum.query('created_month >= "2014-01-01"')).set_title('Количество заметок, созданных в месяц')

In [ ]:
sns.set(rc={'figure.figsize' : (10, 5)})
( 
    sns.lineplot(data=monthes_sum.rolling(3).mean().query('created_month >= "2014-01-01"'), x='created_month', y='created_notes')
    .set_title('Количество заметок, созданных в месяц (сглажено скользящим средним)')
    
)

## Categories

In [ ]:
stacks = pd.read_csv('stacks.csv')
df2 = df.merge(stacks, on='stack')

In [ ]:
stacks

In [ ]:
df2['stack2'].value_counts()

In [ ]:
df2.head()

In [ ]:
categories = list(set(df2.columns).difference(df.columns))

In [ ]:
df2['categories_count'] = df2[categories].sum(axis=1)

In [ ]:
df2.query('categories_count == 0')

In [ ]:
# df_main = df2.query('~professional and ~data')
# (~df_main['operational'] == df_main['reflectional']).mean()

### Reflectional zettlekasten

In [ ]:
df2.query('stack2 == "reflectional"')

In [ ]:
df2[['stack2', 'stack', 'notebook']].drop_duplicates().sort_values(['stack2', 'stack', 'notebook']).iloc[20:50]

## Graphs

In [8]:
notes = pd.read_csv(f'{context_dir}/notes.csv')
links = pd.read_csv(f'{context_dir}/links.csv')

In [10]:
#links['success'].mean()

In [ ]:
#links.query('~success')

In [11]:
#notebooks = pd.read_parquet('../data/full/notebooks')
notebooks2 = pd.read_csv(f'{context_dir}/notebooks2.csv')

In [12]:
notebooks2

,guid,name,stack,authorship,objective,longetivity,it-specific,externality,journals,sensitivity
0,752545ce-5866-4c56-a470-d1d6d66644f4,LJ,Reflections,me,subjective,long,no,more,no,sensitive
1,156433dc-79ae-42ea-8205-d3d82389a5fa,Career,Professional,me,subjective,long,yes,less,no,protected
2,ab4cb370-56bc-4ec3-aa16-5c72cee8221e,Salsa,Archive,me,subjective,long,no,less,no,sensitive
3,8e97a0be-ec6a-47ba-be8a-f9c7952057f8,Dailys,Reflections,me,subjective,long,no,less,yes,private
4,312e634b-a75b-468f-b78a-be9b7dd6a624,northcape,Core,me,subjective,long,no,less,no,sensitive
...,...,...,...,...,...,...,...,...,...,...
117,b0b9addb-51a4-746b-b3e1-5d4ffc257883,LM Reflections,LM Stack,me,subjective,long,no,less,no,protected
118,939ac241-8b0b-49a2-a8eb-fe834d62e273,LM Abstracts,LM Stack,me,subjective,long,no,less,no,protected
119,4b7c5837-fef9-3879-b1a5-f44ff3e3154a,LM Social,LM Stack,me,subjective,long,no,less,no,protected
120,b1ed59ce-625c-cb3f-7dde-dc690e2650f2,LM Cases and Ideas,LM Stack,me,subjective,long,no,less,no,protected


In [16]:
notes2 = notes.merge(notebooks2[['name', 'sensitivity']], left_on='notebook', right_on='name')
                       

In [19]:
notes2

,Unnamed: 0,id,title,created,updated,tagNames,active,contentLength,content,notebook,stack,name,sensitivity
0,0,7ea3327a-4551-435e-bbc9-1e008bcb217a,"""И все тянется долгие годы, почти не начинаясь...",1453534173000,1455744295000,['lj'],True,1686,NaN,LJ,Reflections,LJ,sensitive
1,1,fc444d6a-5344-440a-89bb-a7a322b61049,...,1429566010000,1441494893000,['unfinished'],True,1536,NaN,LJ,Reflections,LJ,sensitive
2,2,950cf7d2-9aca-47d3-b5c0-765ce6bd1e70,1.5X RUB -> 1x RUB,1652074058000,1653283663000,NaN,True,1810,NaN,LJ,Reflections,LJ,sensitive
3,3,66531632-74fc-48fb-968d-e1626d93ae97,13 мая external,1591468199000,1591468419000,['telegram'],True,2687,NaN,LJ,Reflections,LJ,sensitive
4,4,1480d9ef-c192-42b8-a043-456f47baec0e,24 февраля 2023,1677225807000,1677861505000,NaN,True,3228,NaN,LJ,Reflections,LJ,sensitive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9574,9574,2a91a9e4-1a93-e012-49c0-895afda98e43,Кластера в заметках ( Deprecated),1674029492000,1718539955000,"['notion', 'custdev', 'pdm', 'текстовая_карта'...",True,8910,NaN,LM My Notes Meta,LM Stack,LM My Notes Meta,protected
9575,9575,23f8ffc0-5f12-487a-a69e-d39e7798bfe3,Метамодель текстовой карты,1652167284000,1718540268000,"['метамодель', 'типология', 'иерархия', 'текст...",True,18125,NaN,LM My Notes Meta,LM Stack,LM My Notes Meta,protected
9576,9576,f85122a5-a050-2a42-371c-beff90f81308,Мой цифровой сад,1711191984000,1718537859000,"['Проект', 'заметки']",True,677,NaN,LM My Notes Meta,LM Stack,LM My Notes Meta,protected
9577,9577,ddaeebb5-9291-4fec-a891-cbfc29a0457e,Статистика по моим заметкам,1675658756000,1711983754000,NaN,True,1536,NaN,LM My Notes Meta,LM Stack,LM My Notes Meta,protected


In [32]:
KEY = ['notebook', 'sensitivity']
x = (
    links[['from_guid', 'to_guid']]
    .merge(notes2[['id'] + KEY], left_on='from_guid', right_on='id')
    .merge(notes2[['id'] + KEY], left_on='to_guid', right_on='id', suffixes=['_from', '_to'])
    .drop(columns=['from_guid', 'to_guid'])
)

In [34]:
(
    x.assign(dummy=1)
    .pivot_table(index='sensitivity_from', columns='sensitivity_to', values='dummy', aggfunc='count', fill_value=0)
)

sensitivity_to,confidential,private,protected,public,sensitive
sensitivity_from,,,,,
confidential,796,8,174,63,34
private,12,431,350,12,304
protected,141,212,2196,183,501
public,7,0,79,291,8
sensitive,53,217,531,47,778


In [33]:
# x = (
#     links
#     .merge(notes2.rename(columns={'id' : 'from_guid', 'notebook' : 'from_notebook'})[['from_guid', 'from_notebook']], on='from_guid')
#     .merge(notes2.rename(columns={'id' : 'to_guid', 'notebook' : 'to_notebook'})[['to_guid', 'to_notebook']], on='to_guid')
        
#     [['from_notebook', 'to_notebook']]
    
    
#    # .merge(notes[['id', 'notebook']], left_on='to_guid', right_on='id')
# )

## Adj

In [ ]:
(
    x.assign(dummy=1)
    .pivot_table(index='from_notebook', columns='to_notebook', values='dummy', aggfunc='count', fill_value=0)
)

In [ ]:
sns.heatmap(x)

In [ ]:
x.to_csv(f'{context_dir}/adj.csv')

## Links analysis

In [6]:
links = pd.read_csv(f'{context_dir}/links.csv', parse_dates=['ts'])

In [7]:
links.isnull().mean()

from_title    0.000000
from_guid     0.000000
to_guid       0.012749
to_old        0.255881
to_new        0.012749
status        0.000000
ts            0.000000
dtype: float64

In [9]:
links.query('~to_guid.isnull()').isnull().mean()

from_title    0.000000
from_guid     0.000000
to_guid       0.000000
to_old        0.257184
to_new        0.000000
status        0.000000
ts            0.000000
dtype: float64

In [9]:
links['status'].value_counts()

status
success    5498
trash        71
Name: count, dtype: int64

In [10]:
links['processing_time'] = links['ts'].diff()

In [16]:
#notes_status['status'].value_counts()

In [8]:
notes = pd.read_parquet(f'{context_dir}/notes')


In [14]:
pd.read_csv('x.csv').set_index('Name').T.to_csv('x2.csv')

In [11]:
!pip install networkx

In [11]:
import networkx as nx

In [12]:
#notes[['id']]

In [14]:
os.getcwd()

'/home/jovyan/work/notes_analytics'

In [17]:
G = nx.read_graphml('../data/full/notes_main.graphml')

In [18]:
G.number_of_edges()

5569

In [19]:
G.number_of_nodes()

8676

In [20]:
connected_components = list(nx.connected_components(G.to_undirected()))

In [21]:
connected_components

[{'7ea3327a-4551-435e-bbc9-1e008bcb217a'},
 {'fc444d6a-5344-440a-89bb-a7a322b61049'},
 {'950cf7d2-9aca-47d3-b5c0-765ce6bd1e70'},
 {'66531632-74fc-48fb-968d-e1626d93ae97'},
 {'1480d9ef-c192-42b8-a043-456f47baec0e'},
 {'54e16987-b525-46cd-8586-9f592b912625'},
 {'b0ddb2ee-3272-40d8-b7be-78c92c95deeb',
  'a9f6bb8e-be81-446f-939c-40e705925cb1',
  '37a72472-ba4b-de89-1993-96b900a737ff',
  'f29608cb-10a6-4303-ae2a-e34e26f95a2f',
  '5dd54066-4ed1-4378-b5b1-9626118755dd',
  'dab11478-af8e-9fa1-de3c-f79ad9c20010',
  'f05e99dc-b3c0-4fd4-86ae-45e9fa3f8667',
  '08c3a4a7-f8f5-ef29-a5da-78abc0465d2c',
  '26ba7d95-066e-62c4-dc48-5c329b67675c',
  '34a3e966-9f27-4cb8-89ed-9e147ec9da57',
  '54118bc2-faa0-489b-82fb-fe99ace1a533',
  '80c7eb7f-8bd0-42c2-9dc2-253899f3d552',
  '90631cf5-9b87-4a19-a0df-c60d33c28299',
  '693f22a7-08b9-41bb-9acb-2edc02b93c3b',
  'a29a738c-45a1-4be4-994f-360edb16372c',
  'ef0f3cb4-36a8-4674-8ed9-cff8e5f86a32',
  '4748e316-5531-4220-b57a-2d2886ec80ab',
  '663bb2cd-2a70-486c-901e-0

In [91]:
G_main = connected_components[6]

In [73]:
component_lens = [len(c) for c in connected_components]

In [74]:
pd.Series(component_lens).value_counts()

1       6287
2         57
3          9
4          3
5          2
2213       1
13         1
Name: count, dtype: int64

In [75]:
links['from_title'].value_counts().iloc[:20]

from_title
Мои ценности                                     83
Таймлайн важных событий                          60
Личная терапия c Леной                           56
Терапия с Михаилом                               56
Конец 2020 года                                  55
Ресурсы (инфраструктура)                         54
Текстовая карта понятий как метод                47
Моя карта местности (2022)                       42
Моя карта местности (Конец 2021 года)            40
Мои заметки как личный проект                    40
Дела в 2017 году                                 38
Выбор места центра жизненных интересов (2022)    37
Мои личные проекты                               36
Мой пикап как сбор ресурсов (голодные игры)      36
Зима 2023 ☃️                                     35
Делиться чем-то / отдавать                       35
Канал "рефлекнотность"                           34
Познание                                         34
Складывание и рассказывание историй              32
С

In [78]:
#nx.dag_longest_path(G)

In [ ]:
a = sorted(list(nx.simple_cycles(G)), key = lambda s: len(s)) 

In [ ]:
G.to_gr

In [ ]:
G_main

In [95]:
nx.write_gexf(G, 'notes.gexf')

In [ ]:
nx.write_graphml(G_main, 'notes_main.graphml')

## Sensitivity

In [18]:
os.getcwd()

'/home/jovyan/work/notes_analytics'

'../work/data/full/'

In [57]:
notebooks = pd.read_csv(f'{context_dir}/notebooks2.csv')

In [58]:
notebooks['sensitivity'].value_counts()

sensitivity
sensitive       75
confidential    28
public           8
private          6
Name: count, dtype: int64

In [40]:
notebooks['stack'] == 'Techs'

0      False
1      False
2      False
3      False
4      False
       ...  
100    False
101    False
102    False
103    False
104    False
Name: stack, Length: 105, dtype: bool

In [60]:
links = pd.read_csv(f'{context_dir}/links.csv')

In [61]:
notes = pd.read_csv(f'{context_dir}/notes.csv')

In [65]:
notes

,Unnamed: 0,id,title,created,updated,tagNames,active,contentLength,content,notebook,stack
0,0,7ea3327a-4551-435e-bbc9-1e008bcb217a,"""И все тянется долгие годы, почти не начинаясь...",1453534173000,1455744295000,['lj'],True,1686,NaN,LJ,Reflections
1,1,fc444d6a-5344-440a-89bb-a7a322b61049,...,1429566010000,1441494893000,['unfinished'],True,1536,NaN,LJ,Reflections
2,2,950cf7d2-9aca-47d3-b5c0-765ce6bd1e70,1.5X RUB -> 1x RUB,1652074058000,1653283663000,NaN,True,1810,NaN,LJ,Reflections
3,3,66531632-74fc-48fb-968d-e1626d93ae97,13 мая external,1591468199000,1591468419000,['telegram'],True,2687,NaN,LJ,Reflections
4,4,1480d9ef-c192-42b8-a043-456f47baec0e,24 февраля 2023,1677225807000,1677861505000,NaN,True,3228,NaN,LJ,Reflections
...,...,...,...,...,...,...,...,...,...,...,...
9574,9574,2a91a9e4-1a93-e012-49c0-895afda98e43,Кластера в заметках ( Deprecated),1674029492000,1718539955000,"['notion', 'custdev', 'pdm', 'текстовая_карта'...",True,8910,NaN,LM My Notes Meta,LM Stack
9575,9575,23f8ffc0-5f12-487a-a69e-d39e7798bfe3,Метамодель текстовой карты,1652167284000,1718540268000,"['метамодель', 'типология', 'иерархия', 'текст...",True,18125,NaN,LM My Notes Meta,LM Stack
9576,9576,f85122a5-a050-2a42-371c-beff90f81308,Мой цифровой сад,1711191984000,1718537859000,"['Проект', 'заметки']",True,677,NaN,LM My Notes Meta,LM Stack
9577,9577,ddaeebb5-9291-4fec-a891-cbfc29a0457e,Статистика по моим заметкам,1675658756000,1711983754000,NaN,True,1536,NaN,LM My Notes Meta,LM Stack


In [70]:
notes.columns

Index(['Unnamed: 0', 'id', 'title', 'created', 'updated', 'tagNames', 'active',
       'contentLength', 'content', 'notebook', 'stack'],
      dtype='object')

In [75]:
notes_sensitivity = notes.merge(notebooks2, left_on='notebook', right_on='name')[['id', 'sensitivity']]

In [72]:
(
    links
    
    [['title', 'sensitivity']]
    .query('
)

,title,sensitivity
0,Beyond broken pipe,sensitive
1,Beyond broken pipe external,sensitive
2,FW: мой небольшой юбилей в openway,sensitive
3,Untitled,sensitive
4,Untitled,sensitive
...,...,...
7423,Мой цифровой сад,protected
7424,Мой цифровой сад,protected
7425,Статистика по моим заметкам,protected
7426,Статистика по моим заметкам,protected


In [50]:
notebooks[['path', 'sensitivity']]

,path,sensitivity
0,Reflections/LJ.enex,sensitive
1,Professional/Career.enex,sensitive
2,Archive/Salsa.enex,sensitive
3,Reflections/Dailys.enex,private
4,Core/northcape.enex,sensitive
...,...,...
100,Core/Ideas.enex,sensitive
101,Personal Projects/Experiences.enex,sensitive
102,Personal Projects/Two worlds and neuros.enex,sensitive
103,Debug/A.enex,sensitive
